In [1]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
import json


In [2]:
import io
# import pandas as pd
# from google.colab import files
from collections import OrderedDict
import inspect as inspector
import sys
import argparse
# import json
import time
import random
import hashlib

In [3]:
!pip install -q "tensorflow-text"

import requests
import tensorflow as tf
import tensorflow_text as tf_text

In [4]:
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses

!pip install transformers #==2.9.0
!pip install torch  #==2.0.0

# !pip install https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-1.1.0-py3-none-any.whl

!pip install accelerate
# !pip install transformers==2.9.0 
!pip install pytorch_lightning  #==0.7.5
# !pip install datsets transformers[sentencepiece]
!pip install sentencepiece

!pip install Sentencepiece
# !pip install transformers

!pip install accelerate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
data_path = './train.jsonl'
df = pd.read_json(data_path, lines=True)
df_validation = pd.read_json('./validation.jsonl', lines=True)

In [7]:
def concatenate_strings(lst):
    separator = ' '
    return separator.join(lst)

In [8]:
df['targetParagraphs'] = df['targetParagraphs'].apply(concatenate_strings)
df['postText'] = df['postText'].apply(concatenate_strings)
df['spoiler'] = df['spoiler'].apply(concatenate_strings)
df['tags'] = df['tags'].apply(concatenate_strings)

In [9]:
df_validation['targetParagraphs'] = df_validation['targetParagraphs'].apply(concatenate_strings)
df_validation['postText'] = df_validation['postText'].apply(concatenate_strings)
df_validation['spoiler'] = df_validation['spoiler'].apply(concatenate_strings)
df_validation['tags'] = df_validation['tags'].apply(concatenate_strings)

In [10]:
filter_phrase = df['tags'] == "multi"
df_filtered = df[filter_phrase]
# print (df_filtered)

In [11]:
filter_phrase_validation = df_validation['tags'] == "multi"
df_filtered_validation = df_validation[filter_phrase_validation]
# print (df_filtered_validation)

In [12]:
def generate_new_df(df):

    new_df = pd.DataFrame({'tags': df['tags'],'postText':df['postText'],'paragraph' : df['targetParagraphs'] ,'spoiler': df['spoiler']})
    return new_df
    

In [13]:

# nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt



# sklearn

from sklearn.metrics import confusion_matrix, classification_report

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer



# Utility

import re
import string
import pickle

In [14]:
wordLemm = WordNetLemmatizer()
def processed_postText(paragraph):
    # Lower Casing
    paragraph = paragraph.lower()
    # Remove punctuations
    paragraph = paragraph.translate(str.maketrans("","",string.punctuation))
    # Tokenizing words
    tokens = word_tokenize(paragraph)
    # Removing Stop Words
    #final_tokens = [w for w in tokens if w not in stopword]
    # Reducing a word to its word stem using lemmatization
    
    return ' '.join(tokens)

In [15]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
nltk.download('stopwords')
stopword = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
new_df = generate_new_df(df_filtered)

In [18]:
new_df_validation = generate_new_df(df_filtered_validation)

In [19]:
 new_df['paragraph'] = new_df['paragraph'].apply(lambda x: processed_postText(x))
 new_df['postText'] = new_df['postText'].apply(lambda x: processed_postText(x))
# print('Text Preprocessing complete.')

In [20]:
 new_df_validation['paragraph'] = new_df_validation['paragraph'].apply(lambda x: processed_postText(x))
 new_df_validation['postText'] = new_df_validation['postText'].apply(lambda x: processed_postText(x))

In [21]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import torch

In [23]:
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
    import wandb

    wandb.login()

wandb: Currently logged in as: pradhaneva94 (evapradhan). Use `wandb login --relogin` to force relogin


In [24]:
from datetime import datetime

In [25]:
if WANDB_INTEGRATION:
    wandb_run = wandb.init(
        project="bart_wiki_lingua",
        config={
            "per_device_train_batch_size": 4
            # "learning_rate": training_args.learning_rate
            # "dataset": "wiki_lingua " + "english",
        },
    )

    now = datetime.now()
    current_time = now.strftime("%H%M%S")
    wandb_run.name = "run_" + "english" + "_" + current_time

In [ ]:
#FINE TUNING START

In [26]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
from datasets import Dataset
from datasets import dataset_dict

In [28]:
import transformers
from transformers import AutoTokenizer, T5ForConditionalGeneration, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset, load_dataset, load_from_disk, load_metric
import numpy as np
import nltk
nltk.download('punkt')
import torch
import pandas as pd

from transformers import (TrainingArguments, Trainer, EarlyStoppingCallback)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
# max_input = 512
# max_target = 128
# batch_size = 3
model_checkpoints = "t5-small"

In [30]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)

In [31]:
input_text = "Summarize:" + "condition: " + new_df['postText'] + " paragraph: " + new_df["paragraph"]
# input_text = "Summarize:" + new_df["paragraph"]
target_text = new_df["spoiler"]


data_to_process_train = pd.DataFrame({ 'input_ids': input_text, 'labels':target_text})

In [32]:
data_to_process_train

,input_ids,labels
3,Summarize:condition: passion is overrated — 7 ...,Purpose connects us to something bigger and in...
9,Summarize:condition: hot sauce taste test find...,Sriracha Hot Chili Sauce Frank's RedHot Origin...
20,Summarize:condition: human remains found in th...,Southampton County Anjelica 'AJ' Hadsell
43,Summarize:condition: six lessons from the godf...,Daniel Patterson 1) Eat your veggies. 2) Enjoy...
44,Summarize:condition: the fastest growing econo...,1. India 2. Bangladesh 3. China 4. Indonesia 5...
...,...,...
3188,Summarize:condition: medical marijuana bills a...,Georgia Kentucky
3190,Summarize:condition: this guy cheated on the p...,Terry Kniess All of the prices in the show hav...
3191,Summarize:condition: 7 small changes to try th...,Relax with rain sounds. Put pressure on period...
3192,Summarize:condition: 11 of the weirdest foods ...,Cream cheese with a bit of strawberry jam on a...


In [33]:
df1_dict = data_to_process_train.to_dict(orient = 'list')

In [34]:
dict_train_ = Dataset.from_dict(df1_dict)

In [35]:
dict_train_

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 559
})

In [36]:
input_text = "Summarize:" + "condition: " + new_df_validation['postText'] + " paragraph: " + new_df_validation["paragraph"]
# input_text = "Summarize:" + new_df_validation["paragraph"]
target_text = new_df_validation["spoiler"]

data_to_process_validation = pd.DataFrame({ 'input_ids': input_text, 'labels':target_text})

In [37]:
data_to_process_train

,input_ids,labels
3,Summarize:condition: passion is overrated — 7 ...,Purpose connects us to something bigger and in...
9,Summarize:condition: hot sauce taste test find...,Sriracha Hot Chili Sauce Frank's RedHot Origin...
20,Summarize:condition: human remains found in th...,Southampton County Anjelica 'AJ' Hadsell
43,Summarize:condition: six lessons from the godf...,Daniel Patterson 1) Eat your veggies. 2) Enjoy...
44,Summarize:condition: the fastest growing econo...,1. India 2. Bangladesh 3. China 4. Indonesia 5...
...,...,...
3188,Summarize:condition: medical marijuana bills a...,Georgia Kentucky
3190,Summarize:condition: this guy cheated on the p...,Terry Kniess All of the prices in the show hav...
3191,Summarize:condition: 7 small changes to try th...,Relax with rain sounds. Put pressure on period...
3192,Summarize:condition: 11 of the weirdest foods ...,Cream cheese with a bit of strawberry jam on a...


In [38]:
df2_dict = data_to_process_validation.to_dict(orient = 'list')

In [39]:
dict_validation_ = Dataset.from_dict(df2_dict)

In [40]:
dict_validation_

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 143
})

In [41]:
dict_validation_["input_ids"]

['Summarize:condition: why arizona sheriff joe arpaio ’ s fate could hang on a single word paragraph: © reuterslaura segall maricopa county sheriff joe arpaio announces newly launched program aimed at providing security around schools in anthem arizona january 9 2013 phoenix — a single word — intentionally — could transform a court case against phoenixarea sheriff joe arpaio from civil charges to a criminal prosecution in finding arpaio and three of his top commanders in contempt of court on friday us district court judge murray snow said no fewer than 19 times that the sheriff or his aides had repeatedly and intentionally violated judicial orders to stop profiling latinos on may 31 snow will determine the civil penalties and examine whether arpaio the sheriff of arizona ’ s maricopa county will be referred to arizona ’ s us attorney for potential criminal charges arpaio has acknowledged violating snow ’ s order but said it was a mistake in weighing criminal charges snow must decide wh

In [42]:
input_text = "Summarize:" + "condition: " + new_df_validation['postText'] + " paragraph: " + new_df_validation["paragraph"]
# input_text = "Summarize:" + new_df_validation["paragraph"]
target_text = new_df_validation["spoiler"]

data_to_process_test = pd.DataFrame({ 'input_ids': input_text, 'labels':target_text})

In [43]:
data_to_process_test

,input_ids,labels
1,Summarize:condition: why arizona sheriff joe a...,"""intentionally"" could transform a court case a..."
3,Summarize:condition: harry potter alums reunit...,Alan Rickman & Rupert Grint CBGB
10,Summarize:condition: how 1 model prepares for ...,"Elettra Wiedemann extra strength work, so weig..."
13,Summarize:condition: the question former oj si...,Marcia Clark Does she think Simpson really did...
14,Summarize:condition: hollywood legends amazing...,"Mitzi Gaynor Beverly Hills, California"
...,...,...
785,Summarize:condition: 6 transcendental festival...,BOOM – Portugal Meadows in the Mountains – Bul...
788,Summarize:condition: things in the olsen twins...,earned much of their fortune making unintentio...
789,Summarize:condition: 13 pokémon the designers ...,Klefki Rotom Luvdisk Unknown (Unown) Vanilluxe
790,Summarize:condition: 9 things elizabeth taylor...,I don’t pretend to be an ordinary housewife Pe...


In [44]:
df3_dict = data_to_process_test.to_dict(orient = 'list')

In [45]:
dict_test_ = Dataset.from_dict(df3_dict)

In [46]:
dict_test_

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 143
})

In [ ]:
# df1_dict = data_to_process_train.to_dict(orient = 'list')
# df2_dict = data_to_process_validation.to_dict(orient = 'list')
# df3_dict = data_to_process_test.to_dict(orient = 'list')


# dict_train_ = Dataset.from_dict(df1_dict)
# dict_validation_ = Dataset.from_dict(df2_dict)
# dict_test_ = Dataset.from_dict(df3_dict)

In [47]:
dict_all_2 = dataset_dict.DatasetDict({"train":dict_train_, "validation":dict_validation_, "test": dict_test_})

In [48]:
dict_all_2

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 559
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 143
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 143
    })
})

In [ ]:
dict_train_

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1274
})

In [49]:
def preprocess_data(data_to_process):
    data_to_process_input = data_to_process['input_ids']
    data_to_process_target = data_to_process['labels']

    inp = tokenizer(data_to_process_input, padding="max_length", truncation=True, max_length= 1000).input_ids

    with tokenizer.as_target_tokenizer():
      targ = tokenizer(data_to_process_target, padding='max_length', truncation=True, max_length= 500).input_ids
    
    return {"input_ids": inp, "labels": targ}

In [50]:
train_dataset = dict_train_
test_dataset = dict_validation_

train_dataset_processed = train_dataset.map(preprocess_data, batched=True)
test_dataset_processed = test_dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/559 [00:00<?, ? examples/s]

Map:   0%|          | 0/143 [00:00<?, ? examples/s]

In [52]:
train_dataset_processed

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 559
})

In [53]:
test_dataset_processed

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 143
})

In [54]:
from transformers import pipeline
import requests
import pprint
import time
pp = pprint.PrettyPrinter(indent=14)

from transformers import T5Tokenizer, T5ForConditionalGeneration
# import wandb

# tokenizer = T5Tokenizer.from_pretrained("t5-small")
# model = T5ForConditionalGeneration.from_pretrained('t5-small')

In [59]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [55]:
#testing

tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [61]:
#testing
# /content/drive/MyDrive/NLP_Projects/T5_finetuning__multi_results_conditional_all_sizelimited
model_path_saved = '/content/drive/MyDrive/NLP_Projects/T5_finetuning__multi_results_conditional_all_sizelimited/checkpoint-3500'
model_saved = T5ForConditionalGeneration.from_pretrained(model_path_saved)

# tokenizer = AutoTokenizer.from_pretrained(model_saved)

In [57]:
#training

model_name = "t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name, num_labels = 4)

In [ ]:
# bleu_metric = load_metric("bleu")
# meteor_metric = load_metric("meteor")
# bertscore_metric = load_metric("bertscore")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
# #out_1 = []
# #out_2 = []

# def compute_metrics(eval_pred):
#     logits_full, labels = eval_pred

#     #out_1.append(logits_full[0])
#     #out_2.append(logits_full[1])

#     predicted_token_ids = torch.argmax(torch.tensor(logits_full[0]), dim=-1).tolist()
#     decoded_preds = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     decoded_preds_bleu = [pred.split() for pred in decoded_preds]
#     decoded_labels_bleu = [[pred.split()] for pred in decoded_labels]
    
#     bleu_score = bleu_metric.compute(predictions=decoded_preds_bleu, references= decoded_labels_bleu)
#     meteor_score = meteor_metric.compute(predictions=decoded_preds, references=decoded_labels)
#     bertscore_score = bertscore_metric.compute(predictions=decoded_preds, references=decoded_labels, lang = 'en')
    
#     return {"bleu": bleu_score["bleu"], "meteor": meteor_score["meteor"], "bertscore": np.mean(bertscore_score["f1"])}

In [58]:
training_args = TrainingArguments(
    output_dir=f"/content/drive/MyDrive/NLP_Projects/T5_finetuning__multi_results_conditional_all_sizelimited",
    num_train_epochs=25,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    # gradient_accumulation_steps=2,
    evaluation_strategy="epoch",
    logging_steps=100,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=3,
    # predict_with_generate=True,
    fp16=True,
    # save_strategy="epoch"

)




In [59]:
#training

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_processed,
    eval_dataset=test_dataset_processed,
    tokenizer=tokenizer,
    # data_collator=data_collator,
    # compute_metrics=compute_metrics
)

In [60]:
trainer.train()


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,2.691800,0.384430
2,0.411500,0.288122
3,0.266800,0.262764
4,0.235800,0.247869
5,0.221400,0.238105
6,0.219100,0.231079
7,0.208900,0.221758
8,0.193800,0.218748
9,0.193500,0.213260
10,0.180400,0.207617


TrainOutput(global_step=3500, training_loss=0.2639267907823835, metrics={'train_runtime': 1307.3241, 'train_samples_per_second': 10.69, 'train_steps_per_second': 2.677, 'total_flos': 3694143897600000.0, 'train_loss': 0.2639267907823835, 'epoch': 25.0})

In [62]:
metrics = trainer.evaluate()


In [63]:
metrics

{'eval_loss': 0.18734778463840485,
 'eval_runtime': 4.2572,
 'eval_samples_per_second': 33.59,
 'eval_steps_per_second': 8.456,
 'epoch': 25.0}

In [ ]:
# model_path_saved = '/content/drive/MyDrive/T5_finetuning__multi_results_conditional_all_sizelimited/checkpoint-7500'
# model_saved = T5ForConditionalGeneration.from_pretrained(model_path_saved)

In [64]:
training_args = TrainingArguments(
    output_dir=f"/content/drive/MyDrive/T5_finetuning__multi_results_conditional_all_sizelimited_testing",
    num_train_epochs=25,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    # gradient_accumulation_steps=2,
    evaluation_strategy="epoch",
    logging_steps=100,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=3,
    # predict_with_generate=True,
    fp16=True,
    # save_strategy="epoch"

)

In [65]:
trainer_saved = Trainer(
    model=model_saved,
    args=training_args,
    train_dataset=train_dataset_processed,
    eval_dataset=test_dataset_processed,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics
)

# predictions_output = trainer_saved.predict(test_dataset_processed.select(range(len(test_dataset_processed))))
# logits_full = predictions_output.predictions
# predicted_token_ids = torch.argmax(torch.tensor(logits_full[0]), dim=-1).tolist()
# decoded_preds = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)

# result_frame = pd.DataFrame()
# result_frame['actual_spoiler'] = test_dataset_processed["labels"]
# result_frame['gen_spoiler'] = decoded_preds

In [66]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples,
        padding="max_length",
        truncation=True,
        max_length=1000,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


    # predictions_output = trainer_saved.predict(test_samples)
    # logits_full = predictions_output.predictions
    # predicted_token_ids = torch.argmax(torch.tensor(logits_full[0]), dim=-1).tolist()
    # decoded_preds = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)
    # return decoded_preds
    # result_frame = pd.DataFrame()
    # result_frame['actual_spoiler'] = test_dataset_processed["labels"]
    # result_frame['gen_spoiler'] = decoded_preds

In [ ]:
train_dataset_processed["labels"]

In [67]:
# model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)
summaries_after_tuning_all = []

def generate_batch(lst, batch_size):
    """  Yields batch of specified size """
    for i in range(0, len(lst), 
                   batch_size):
        yield lst[i : i + batch_size]
        # summaries_after_tuning = generate_summary(test_samples, model)[1]
        # summaries_after_tuning_all.append(summaries_after_tuning)


# test_samples = dict_validation_.select(range())
# test_samples = generate_batch(dict_validation_["input_ids"], 16)
test_samples = generate_batch(dict_validation_["input_ids"], 16)

# # summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]

for batch in test_samples:
  # print(batch)
  for sample in batch:
    # print(sample)
    # print("\n")
    summaries_after_tuning = generate_summary(sample, model_saved)[1]
    summaries_after_tuning_all.append(summaries_after_tuning)
    # print(summaries_after_tuning)
    # print("\n")

In [ ]:
# dataframe = result_frame

In [69]:
summaries_after_tuning_all

[[''],
 [''],
 [''],
 [''],
 [''],
 [''],
 ['1. Create your own cleaner 2. Clean shelves in 30 minutes when was the last time you removed the'],
 [''],
 [''],
 [''],
 ['1. mouthwash for your feet if you have toenail or foot fungus'],
 [''],
 [''],
 ['1. Washing your face without a doubt 1. Washing your face without a doubt 2.'],
 [''],
 [''],
 [''],
 [''],
 [''],
 ['1. Mylan has patent protection that lasts through 2025. Epinephrine'],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 ['1. Dr Lakisha Jenkins Master herbalist 2. Rachel K.'],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 ['1. Coldweather clothing 2. Backtoschool supplies 2. Backtoschool supplies 2. Backtoschool supplies'],
 [''],
 [''],
 ['1. Jessica 2. Amy 3. Jade 4. Rebecca 5 Charlotte 7'],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''

'Einige der plot elements are so disturbing that they make him feel sick.'

In [68]:
stories_list =[]
for i in range(len(summaries_after_tuning_all)):
  for j in range(len(summaries_after_tuning_all[i])):
    summary=summaries_after_tuning_all[i][j]
    stories_list.append(summary)


print(stories_list)

['', '', '', '', '', '', '1. Create your own cleaner 2. Clean shelves in 30 minutes when was the last time you removed the', '', '', '', '1. mouthwash for your feet if you have toenail or foot fungus', '', '', '1. Washing your face without a doubt 1. Washing your face without a doubt 2.', '', '', '', '', '', '1. Mylan has patent protection that lasts through 2025. Epinephrine', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '1. Dr Lakisha Jenkins Master herbalist 2. Rachel K.', '', '', '', '', '', '', '1. Coldweather clothing 2. Backtoschool supplies 2. Backtoschool supplies 2. Backtoschool supplies', '', '', '1. Jessica 2. Amy 3. Jade 4. Rebecca 5 Charlotte 7', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '1. Get an education at least this one will be free investing is understandably scary until you learn', '', '', '', '1. Eat legumes legumes and pulses 4. Us

In [ ]:
# type(new_df['spoiler'][0:40])

In [ ]:
# type(stories_list[0:40])

list

In [ ]:
# stories_list_series = pd.Series((v for v in stories_list))

In [70]:
dataframe = pd.DataFrame({ 'postText': new_df_validation["postText"], 'actual_spoiler':test_dataset['labels'], 'gen_spoiler':stories_list })

In [71]:
dataframe

,postText,actual_spoiler,gen_spoiler
1,why arizona sheriff joe arpaio ’ s fate could ...,"""intentionally"" could transform a court case a...",
3,harry potter alums reunite for new movie,Alan Rickman & Rupert Grint CBGB,
10,how 1 model prepares for lingerie shoots,"Elettra Wiedemann extra strength work, so weig...",
13,the question former oj simpson prosecutor hate...,Marcia Clark Does she think Simpson really did...,
14,hollywood legends amazing home is on the market,"Mitzi Gaynor Beverly Hills, California",
...,...,...,...
785,6 transcendental festivals out in the middle o...,BOOM – Portugal Meadows in the Mountains – Bul...,
788,things in the olsen twins lives that make no s...,earned much of their fortune making unintentio...,
789,13 pokémon the designers just gave up on,Klefki Rotom Luvdisk Unknown (Unown) Vanilluxe,
790,9 things elizabeth taylor taught us about the ...,I don’t pretend to be an ordinary housewife Pe...,


In [72]:
file = open("T5_conditional_finetuning_no_tags_multi_results_without_postText.txt", "w")

#convert variable to string
# str_1 = repr(dataframe)
str2 = dataframe.to_json()
file.write("T5_conditional_finetuning_no_tags_multi_results = " + str2 + "\n")

#close file
file.close()

In [73]:
if WANDB_INTEGRATION:
    wandb_run.finish()

eval/loss,█▅▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▄▅▆▅▅▅▅▆▅▆▅▆▆▅▆▅▅▅▅▆▆▅▆▆█▁
eval/samples_per_second,▅▄▃▄▄▄▄▃▄▃▄▃▃▄▃▄▄▄▄▃▃▄▃▃▁█
eval/steps_per_second,▅▄▃▄▄▄▄▃▄▃▄▃▃▄▃▄▄▄▄▃▃▄▃▃▁█
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/learning_rate,███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [ ]:
#EVALUATION METRICS

In [74]:
import nltk
import nltk.translate.bleu_score as bleu

import math
import numpy
import os

try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')

In [75]:
bleu_scores = []
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  score_ref = bleu.sentence_bleu([dataframe.iloc[i]["gen_spoiler"]], dataframe.iloc[i]["actual_spoiler"])
  bleu_scores.append(score_ref)



In [76]:
bleu_scores

[0,
 0,
 0,
 0,
 0,
 0,
 0.18751841292984597,
 0,
 0,
 0,
 1.4385877528520696e-155,
 0,
 0,
 0.23104035621498045,
 0,
 0,
 0,
 0,
 0,
 0.21184366433811433,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.1905732212376011,
 0,
 0,
 0,
 0,
 0,
 0,
 0.3898433770295491,
 0,
 0,
 0.9273397041322389,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.2650179784476125,
 0,
 0,
 0,
 0.35891427787336616,
 0,
 0,
 0,
 0,
 0,
 0.04489421571414015,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.09300616718241356,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [77]:
bleu_average = round(sum(bleu_scores) / len(bleu_scores), 2)
bleu_average

0.02

In [78]:
bleu_min = min(bleu_scores)
bleu_min

0

In [79]:
bleu_max = max(bleu_scores)
bleu_max

0.9273397041322389

In [80]:
bleu_scores.sort()
mid = len(bleu_scores) // 2
res = (bleu_scores[mid] + bleu_scores[~mid]) / 2
bleu_median = res
bleu_median

0.0

In [81]:
import sys
from nltk.translate.meteor_score import meteor_score

In [82]:
meteor_scores = []
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  gen_spoiler_strip = dataframe.iloc[i]["gen_spoiler"].strip()

  ref = word_tokenize(gen_spoiler_strip)
  hypo = word_tokenize(dataframe.iloc[i]["actual_spoiler"].strip())
  meteor = nltk.translate.meteor_score.meteor_score([ref], hypo)


  # meteor = nltk.translate.meteor_score.meteor_score([gen_spoiler_strip], dataframe.iloc[i]["actual_spoiler"].strip())
  # meteor = round(meteor_score([dataframe.iloc[i]["gen_spoiler"]], dataframe.iloc[i]["actual_spoiler"]), 2) # list of references
  meteor_scores.append(meteor)



In [83]:
meteor_scores

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.22167968750000003,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.35584293238556075,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6647110472541508,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5345161290322582,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.24390243902439024,
 0.0,
 0.0,
 0.7632653061224489,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.27259475218658896,
 0.0,
 0.0,
 0.0,
 0.5511111111111111,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.8522282445046673,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.35606744956338454,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,

In [84]:
meteor_scores_average = round(sum(meteor_scores) / len(meteor_scores), 2)
meteor_scores_average

0.03

In [85]:
meteor_scores_max = max(meteor_scores)
meteor_scores_max

0.8522282445046673

In [86]:
meteor_scores_min = min(meteor_scores)
meteor_scores_min

0.0

In [87]:
meteor_scores.sort()
mid = len(meteor_scores) // 2
res = (meteor_scores[mid] + meteor_scores[~mid]) / 2
meteor_scores_median = res
meteor_scores_median

0.0

In [88]:
drive.flush_and_unmount()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'drive' is not defined

In [89]:
!pip install bert-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [90]:
from bert_score import score

In [ ]:
!git clone https://github.com/Tiiiger/bert_score.git

Cloning into 'bert_score'...
remote: Enumerating objects: 993, done.
remote: Counting objects: 100% (355/355), done.
remote: Compressing objects: 100% (156/156), done.
remote: Total 993 (delta 219), reused 322 (delta 198), pack-reused 638
Receiving objects: 100% (993/993), 1.34 MiB | 10.33 MiB/s, done.
Resolving deltas: 100% (534/534), done.


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [91]:
!pip install bert_score
bertscore_metric = load_metric('bertscore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [92]:
dataframe.iloc[0]

postText          why arizona sheriff joe arpaio ’ s fate could ...
actual_spoiler    "intentionally" could transform a court case a...
gen_spoiler                                                        
Name: 1, dtype: object

In [93]:
dataframe.count()[0]

143

In [94]:
bert_scores_all = []
for i in range(dataframe.count()[0]):
# for data_results in dataframe[0]:
  gen_spoiler_strip = dataframe.iloc[i]["gen_spoiler"]
  actual_spoiler_strip = dataframe.iloc[i]["actual_spoiler"]

  ref = gen_spoiler_strip
  hypo = actual_spoiler_strip

  counter = 0
  for c in hypo: # traverse the string “educative”
    counter+=1 #increment the counter
  # print (counter)

  # print(len(hypo))
  # print("\n")

  counter2 = 0
  for c in ref: # traverse the string “educative”
    counter2+=1 #increment the counter
  # print (counter2)

  len1 = min(counter, counter2)
  # print(hypo[:len1])
  # print("\n")

  if len1 > 0:
    bert_scores = bertscore_metric.compute(predictions=hypo[:len1], references=ref[:len1], lang="en")
    bert_scores_all.append(bert_scores)
  else:
    bert_scores_all.append(0)
# print(bert_scores)
# Normally, we use the f1-score attribute
# print(bert_scores['f1'])

In [95]:
bert_scores_all

[0,
 0,
 0,
 0,
 0,
 0,
 {'precision': [0.9411759376525879,
   0.8340387940406799,
   0.0,
   0.8942504525184631,
   0.9988315105438232,
   0.8955549597740173,
   0.999210774898529,
   0.8971014022827148,
   0.0,
   0.0,
   0.9741451740264893,
   0.8944270610809326,
   0.9933202266693115,
   0.0,
   0.0,
   0.9988486766815186,
   0.8999120593070984,
   0.992722749710083,
   0.0,
   0.911933183670044,
   0.9995307922363281,
   0.0,
   0.9984025955200195,
   0.9931254982948303,
   0.8955549597740173,
   0.9918492436408997,
   0.0,
   0.0,
   0.8351538777351379,
   0.0,
   0.8942504525184631,
   0.0,
   0.925399124622345,
   0.940531313419342,
   0.9075086116790771,
   0.0,
   0.9826475381851196,
   0.9495258331298828,
   0.925399124622345,
   0.0,
   0.9985170364379883,
   0.9075086116790771,
   0.0,
   0.0,
   0.0,
   0.9935994148254395,
   0.0,
   0.983052670955658,
   0.9265478253364563,
   0.0,
   0.9982914924621582,
   0.9987497329711914,
   0.9282351732254028,
   0.9993563294410706

In [96]:
# type(bert_scores_all)
counter2 = 0
for i in bert_scores_all:
 
    # incrementing counter
    counter2 = counter2 + 1

counter2

143

In [97]:
bert_sum = 0
# counter = 0
for i in range(counter2 - 1):
  berts_all = bert_scores_all[i]
  # print(berts_all["f1"])
  # print("\n")
  # count = 0
  if isinstance(berts_all, dict):
    if "f1" in berts_all:
      for b in berts_all["f1"]:
        # print(b)
        # print("\n")
        bert_sum += b
      #   counter = counter + 1
  # else:
  #   print "Key does not exist"

  # print(len(berts_all["f1"]))
#   for berts in range(len(berts_all["f1"])):

#     bert_sum += berts
#     counter += counter

bertscore_average = round(bert_sum / (counter2 -1), 2)
print(bertscore_average)

3.91
